# Rainbow options with Integration
In this Notebook we will go through the implementation of the Integration Method for the rainbow option presented in https://arxiv.org/pdf/2402.05574.pdf using classiq platform QMOD language

## Data Definitions


In [2]:
import numpy as np
import scipy

K = 190
S0 = [193.97, 189.12]
dt = 250

COV = np.array([[0.000335, 0.000257], [0.000257, 0.000418]])
MU_LOG_RET = np.array([0.00050963, 0.00062552])

MU = MU_LOG_RET * dt
CHOLESKY = np.linalg.cholesky(COV) * np.sqrt(dt)
SCALING_FACTOR = 1 / CHOLESKY[0, 0]


NUM_QUBITS = 2
NUM_ASSETS = 2

## Gaussian State preparation

In [3]:
def gaussian_discretization(num_qubits, mu=0, sigma=1, stds_around_mean_to_include=3):
    lower = mu - stds_around_mean_to_include * sigma
    upper = mu + stds_around_mean_to_include * sigma
    num_of_bins = 2**num_qubits
    sample_points = np.linspace(lower, upper, num_of_bins + 1)

    def single_gaussian(x: np.ndarray, _mu: float, _sigma: float) -> np.ndarray:
        cdf = scipy.stats.norm.cdf(x, loc=_mu, scale=_sigma)
        return cdf[1:] - cdf[0:-1]

    non_normalized_pmf = (single_gaussian(sample_points, mu, sigma),)
    real_probs = non_normalized_pmf / np.sum(non_normalized_pmf)
    return sample_points[:-1], real_probs[0].tolist()


grid_points, probabilities = gaussian_discretization(NUM_QUBITS)

STEP_X = grid_points[1] - grid_points[0]
MIN_X = grid_points[0]

### SANITY CHECK

The process must be stopped if the strike price $K$ is greater than the maximum value reacheable by the assets during the simulation, to avoid meaningless results. The payoff is $0$ in this case, so there is no need to simulate.

In [4]:
from IPython.display import Markdown

if K >= max(S0*np.exp(np.dot(CHOLESKY,[grid_points[-1]]*2) + MU)):
    display(Markdown('<font color=\'red\'> K always greater than the maximum asset values. Stop the run, the payoff is 0</font>'))

### CLASSICAL SIMULATION

In [5]:
from itertools import product
binary_combinations = list(product(range(2**NUM_QUBITS),repeat=2))

In [6]:
import pandas as pd

simulation = pd.DataFrame.from_dict({"quantum_samples": list(range(len(grid_points))) ,"classical_samples": grid_points, "probabilities": probabilities} )

In [7]:
sim = pd.DataFrame(binary_combinations)
sim = sim.merge(simulation, left_on=0, right_on="quantum_samples")
sim = sim.merge(simulation, left_on=1, right_on="quantum_samples")

In [8]:
sim["probabilities_xy"] = sim["probabilities_x"]*sim["probabilities_y"]
sim["perturbed_sample_x"] = sim["classical_samples_x"]*CHOLESKY[0,0]
sim["perturbed_sample_y"] = (sim["classical_samples_x"]*CHOLESKY[1,0]) + (sim["classical_samples_y"]*CHOLESKY[1,1])
sim["ret_x"] = sim["perturbed_sample_x"] + MU[0]
sim["ret_y"] = sim["perturbed_sample_y"] + MU[1]
sim["exp_x"] = np.exp(sim["ret_x"])
sim["exp_y"] = np.exp(sim["ret_y"])
sim["asset_x"] = sim["exp_x"]*S0[0]
sim["asset_y"] = sim["exp_y"]*S0[1]
sim["max_asset"] = sim[["asset_x", "asset_y"]].max(axis=1)
sim["payoff"] = sim["max_asset"] - K
sim.loc[sim["payoff"]<0, "payoff" ] = 0

In [9]:
sim

,0,1,quantum_samples_x,classical_samples_x,probabilities_x,quantum_samples_y,classical_samples_y,probabilities_y,probabilities_xy,perturbed_sample_x,perturbed_sample_y,ret_x,ret_y,exp_x,exp_y,asset_x,asset_y,max_asset,payoff
0,0,0,0,-3.0,0.065635,0,-3.0,0.065635,0.004308,-0.868188,-1.370945,-0.740780,-1.214565,0.476742,0.296839,92.473604,56.138218,92.473604,0.000000
1,1,0,1,-1.5,0.434365,0,-3.0,0.065635,0.028509,-0.434094,-1.037924,-0.306686,-0.881544,0.735881,0.414143,142.738905,78.322750,142.738905,0.000000
2,2,0,2,0.0,0.434365,0,-3.0,0.065635,0.028509,0.000000,-0.704902,0.127407,-0.548522,1.135880,0.577803,220.326604,109.274099,220.326604,30.326604
3,3,0,3,1.5,0.065635,0,-3.0,0.065635,0.004308,0.434094,-0.371881,0.561501,-0.215501,1.753303,0.806137,340.088165,152.456707,340.088165,150.088165
4,0,1,0,-3.0,0.065635,1,-1.5,0.434365,0.028509,-0.868188,-1.018494,-0.740780,-0.862114,0.476742,0.422269,92.473604,79.859433,92.473604,0.000000
5,1,1,1,-1.5,0.434365,1,-1.5,0.434365,0.188673,-0.434094,-0.685472,-0.306686,-0.529092,0.735881,0.589139,142.738905,111.418043,142.738905,0.000000
6,2,1,2,0.0,0.434365,1,-1.5,0.434365,0.188673,0.000000,-0.352451,0.127407,-0.196071,1.135880,0.821954,220.326604,155.447890,220.326604,30.326604
7,3,1,3,1.5,0.065635,1,-1.5,0.434365,0.028509,0.434094,-0.019430,0.561501,0.136950,1.753303,1.146771,340.088165,216.877318,340.088165,150.088165
8,0,2,0,-3.0,0.065635,2,0.0,0.434365,0.028509,-0.868188,-0.666043,-0.740780,-0.509663,0.476742,0.600698,92.473604,113.604052,113.604052,0.000000
9,1,2,1,-1.5,0.434365,2,0.0,0.434365,0.188673,-0.434094,-0.333021,-0.306686,-0.176641,0.735881,0.838080,142.738905,158.497759,158.497759,0.000000


In [10]:
sum(sim["payoff"]*sim["probabilities_xy"])

27.29956087671723

## Maximum Computation

### Precision utils

In [11]:
FRAC_PLACES = 2

def round_factor(a):
    precision_factor = 2 ** (FRAC_PLACES)
    return round(a * precision_factor) / precision_factor


def floor_factor(a):
    precision_factor = 2 ** (FRAC_PLACES)
    return np.floor(a * precision_factor) / precision_factor

### Affine and maximum arithmetic definitions

In [12]:
from functools import reduce

from classiq import QNum, get_expression_numeric_attributes, qfunc, Output
from classiq.qmod.symbolic import max as qmax

a = STEP_X / SCALING_FACTOR
b = np.log(S0[0]) + MU[0] + MIN_X * CHOLESKY[0].sum()


def get_affine_formula(assets, i):
    return reduce(
        lambda x, y: x + y,
        [
            assets[j] * round_factor(SCALING_FACTOR * CHOLESKY[i, j])
            for j in range(NUM_ASSETS)
            if CHOLESKY[i, j]
        ],
    )


c = (
    SCALING_FACTOR
    * (
        np.log(S0[1])
        + MU[1]
        - (np.log(S0[0]) + MU[0])
        + MIN_X * sum(CHOLESKY[1] - CHOLESKY[0])
    )
    / (STEP_X)
)

c = round_factor(c)


def calculate_max_reg_type():
    x1 = QNum("x1", NUM_QUBITS, False, 0)
    x2 = QNum("x2", NUM_QUBITS, False, 0)
    expr = qmax(get_affine_formula([x1, x2], 0), get_affine_formula([x1, x2], 1) + c)
    size_in_bits, sign, fraction_digits = get_expression_numeric_attributes(
        [x1, x2], expr
    )
    return size_in_bits, fraction_digits


MAX_NUM_QUBITS = calculate_max_reg_type()[0]
MAX_FRAC_PLACES = calculate_max_reg_type()[1]

In [13]:
@qfunc
def affine_max(x1: QNum, x2: QNum, res: Output[QNum]):
    res |= qmax(get_affine_formula([x1, x2], 0), get_affine_formula([x1, x2], 1) + c)

### CLASSICAL SIMULATION (with arithmetic precision)

In [14]:
approximated_simulation = sim.iloc[:,:9].copy()

In [15]:
approximated_simulation

,0,1,quantum_samples_x,classical_samples_x,probabilities_x,quantum_samples_y,classical_samples_y,probabilities_y,probabilities_xy
0,0,0,0,-3.0,0.065635,0,-3.0,0.065635,0.004308
1,1,0,1,-1.5,0.434365,0,-3.0,0.065635,0.028509
2,2,0,2,0.0,0.434365,0,-3.0,0.065635,0.028509
3,3,0,3,1.5,0.065635,0,-3.0,0.065635,0.004308
4,0,1,0,-3.0,0.065635,1,-1.5,0.434365,0.028509
5,1,1,1,-1.5,0.434365,1,-1.5,0.434365,0.188673
6,2,1,2,0.0,0.434365,1,-1.5,0.434365,0.188673
7,3,1,3,1.5,0.065635,1,-1.5,0.434365,0.028509
8,0,2,0,-3.0,0.065635,2,0.0,0.434365,0.028509
9,1,2,1,-1.5,0.434365,2,0.0,0.434365,0.188673


In [16]:
approximated_simulation["zx_a"] = CHOLESKY[0,0]*SCALING_FACTOR
approximated_simulation["zy_a"] = CHOLESKY[1,0]*SCALING_FACTOR
approximated_simulation["zy_b"] = SCALING_FACTOR*CHOLESKY[1,1]
approximated_simulation["zy_c"] = SCALING_FACTOR * (np.log(S0[1]) + MU[1] - (np.log(S0[0]) + MU[0]) + MIN_X*sum(CHOLESKY[1] - CHOLESKY[0]))/(STEP_X)
approximated_simulation[["approx_zx_a","approx_zy_a", "approx_zy_b", "approx_zy_c"]]= approximated_simulation[["zx_a","zy_a", "zy_b", "zy_c"] ].applymap(lambda x : round_factor(x))
approximated_simulation["zx"] = approximated_simulation["approx_zx_a"]*approximated_simulation["quantum_samples_x"]
approximated_simulation["zy"] = (approximated_simulation["approx_zy_a"]*approximated_simulation["quantum_samples_x"])+(approximated_simulation["approx_zy_b"]*approximated_simulation["quantum_samples_y"])+ approximated_simulation["approx_zy_c"]
approximated_simulation["z"] = approximated_simulation[["zx", "zy"]].max(axis=1)
approximated_simulation["max_asset"] = S0[0] * np.exp((STEP_X / SCALING_FACTOR ) * approximated_simulation["z"] + (MU[0]+MIN_X*CHOLESKY[0].sum()))
mask = sim["max_asset"] != approximated_simulation["max_asset"]
approximated_simulation["payoff"] = approximated_simulation["max_asset"] - K
approximated_simulation.loc[approximated_simulation["payoff"]<0, "payoff" ] = 0
sum(approximated_simulation["payoff"]*approximated_simulation["probabilities_xy"])

24.932955958535757

## Integration Method

In [17]:
from classiq import QBit, prepare_exponential_state, bind

@qfunc
def integrator(x: QNum, ref: QNum, res: QBit) -> None:
    exp_rate = (1 / (2**x.fraction_digits)) * a
    prepare_exponential_state(-exp_rate, ref)
    x_uint = QNum("x_uint", x.size, False, 0)
    bind(x, x_uint)
    res ^= x_uint >= ref
    bind(x_uint, x)

In [18]:
from classiq import control, RY
from classiq.qmod.symbolic import asin, exp, sqrt

def get_strike_price_theta_integration(x: QNum):
    exp_rate = (1 / (2**x.fraction_digits)) * a
    B = (exp((2**x.size) * exp_rate) - 1) / exp(exp_rate)
    A = 1 / exp(exp_rate)
    C = S0[0] * exp((MU[0] + MIN_X * CHOLESKY[0].sum()))
    return 2 * asin(sqrt((K - (C * A)) / (C * B)))

@qfunc
def integration_load_amplitudes(
    geq_reg: QNum, max_reg: QNum, integrator_reg: QNum, ind_reg: QBit
):
    control(geq_reg == 1, lambda: integrator(max_reg, integrator_reg, ind_reg))
    strike_price_theta = get_strike_price_theta_integration(max_reg)
    control(geq_reg == 0, lambda: RY(strike_price_theta, ind_reg))

In [19]:
@qfunc
def asset_geq_strike_price(
    x: QNum,
    res: Output[QBit],
) -> None:
    a = STEP_X / SCALING_FACTOR
    b = np.log(S0[0]) + MU[0] + MIN_X * CHOLESKY[0].sum()
    COMP_VALUE = (np.log(K) - b) / a
    res |= x > floor_factor(COMP_VALUE)


In [20]:
from classiq import within_apply

@qfunc
def integration_payoff(max_reg: QNum, integrator_reg: QNum, ind_reg: QBit):
    geq_reg = QBit("geq_reg")
    within_apply(
        lambda: asset_geq_strike_price(max_reg, geq_reg),
        lambda: integration_load_amplitudes(geq_reg, max_reg, integrator_reg, ind_reg),
    )

In [21]:
from classiq import Constraints, create_model, synthesize, show, allocate, allocate_num, inplace_prepare_state

@qfunc
def rainbow_integration(
    x1: QNum, 
    x2: QNum,
    integrator_reg: QNum,
    ind_reg: QBit,
) -> None:
    inplace_prepare_state(probabilities, 0, x1)
    inplace_prepare_state(probabilities, 0, x2)
    max_out = QNum("max_out")
    within_apply(
        lambda: affine_max(x1, x2, max_out),
        lambda: integration_payoff(max_out, integrator_reg, ind_reg),
    )


@qfunc
def main(
    x1: Output[QNum], 
    x2: Output[QNum],
    integrator_reg: Output[QNum],
    ind_reg: Output[QBit],
) -> None:
    allocate_num(MAX_NUM_QUBITS, False, MAX_FRAC_PLACES, integrator_reg)
    allocate(NUM_QUBITS, x1)
    allocate(NUM_QUBITS, x2)
    allocate(1, ind_reg)
    rainbow_integration(x1, x2, integrator_reg, ind_reg)


constraints = Constraints(max_width=23)
qmod = create_model(main, constraints=constraints)
print("Starting synthesis")
qprog = synthesize(qmod)
show(qprog)

Starting synthesis
Opening: https://platform.classiq.io/circuit/f60c85f4-40d0-4f6c-ad89-a3247084c9e7?version=0.40.0


## IQAE algorithm

In [22]:
from classiq import cfunc, Z
from classiq.qmod.builtins.classical_execution_primitives import iqae, save


@qfunc
def qmci_oracle(ind: QBit):
    Z(ind)

@cfunc
def cmain():
    iqae_res = iqae(epsilon=0.05, alpha=0.1)
    save({"iqae_res": iqae_res})

In [23]:
from classiq import CInt, QCallable, grover_operator, power, QArray

@qfunc
def grover_algorithm(
    k: CInt,
    oracle_operand: QCallable[QArray[QBit]],
    sp_operand: QCallable[QArray[QBit]],
    x: QArray[QBit],
):
    sp_operand(x)
    power(k, lambda: grover_operator(oracle_operand, sp_operand, x))

In [24]:
def get_main():
    @qfunc
    def main(
        k: CInt,
        ind_reg: Output[QBit],
    ) -> None:
        full_reg = QArray("full_reg")
        allocate(2 * NUM_QUBITS + MAX_NUM_QUBITS + 1, full_reg)
        grover_algorithm(
            k,
            lambda x: qmci_oracle(x[x.len - 1]),
            lambda x: rainbow_integration(
                x[0:NUM_QUBITS],
                x[NUM_QUBITS : 2 * NUM_QUBITS],
                x[2 * NUM_QUBITS : 2 * NUM_QUBITS + MAX_NUM_QUBITS],
                x[x.len - 1],
            ),
            full_reg,
        )
        state_reg = QArray("state_reg")
        bind(full_reg, [state_reg, ind_reg])

    return main

In [25]:
from classiq import execute
from classiq.execution import ExecutionPreferences

def synthesize_and_execute(post_process):
    constraints = Constraints(max_width=25)
    qmod = create_model(
        get_main(),
        constraints=constraints,
        classical_execution_function=cmain,
    )
    print("Starting synthesis")
    qprog = synthesize(qmod)
    show(qprog)
    print("Starting execution")
    res = execute(qprog).result()
    iqae_res = res[0].value
    print("raw iqae results:", iqae_res.estimation, iqae_res.confidence_interval)
    parsed_res = post_process(res[0].value)

    return (qmod, qprog, iqae_res, parsed_res)

## Post Process

### Post process integration method

In [26]:
def parse_result_integration(iqae_res):
    exp_rate = (1 / (2**MAX_FRAC_PLACES)) * a
    B = (np.exp((2**MAX_NUM_QUBITS) * exp_rate) - 1) / np.exp(exp_rate)
    A = 1 / np.exp(exp_rate)
    C = S0[0] * np.exp((MU[0] + MIN_X * CHOLESKY[0].sum()))

    option_value = (iqae_res.estimation * (C * B)) + (C * A) - K
    confidence_interval = (
        (np.array(iqae_res.confidence_interval) * (C * B)) + (C * A) - K
    )
    return (option_value, confidence_interval)

# Run Integration method

In [27]:
print("Integration method")
qmod_integration, qprog_integration, iqae_res_integration, parsed_res_integration = (
    synthesize_and_execute(parse_result_integration)
)
print("option estimated value:", parsed_res_integration)

Integration method
Starting synthesis
Opening: https://platform.classiq.io/circuit/bca783d8-9143-4f04-bcb1-28c0951167d6?version=0.40.0
Starting execution
raw iqae results: 0.05158204154718703 (0.04786286061181038, 0.05530122248256367)
option estimated value: (26.589561198112648, array([16.95481648, 36.22430592]))
